# AllSides sources & bias crawler

Get and save a list of rated news sources as left or right and in between.

A CSV file will be created with the following columns:

- Source
- Label
- Agree
- Disagree
- Publisher URL
- Publisher site

In [1]:
!ipython -m pip install aiohttp bs4 requests

]0;IPython: research/crawlersRequirement already satisfied: aiohttp in /Users/cmin/anaconda3/lib/python3.7/site-packages (3.6.2)


In [2]:
import asyncio
import csv
import logging
import re
import urllib.parse as urlparse

import aiohttp

import bs4
import requests

In [7]:
url_tpl = "https://www.allsides.com/media-bias/media-bias-ratings?field_featured_bias_rating_value=All&field_news_source_type_tid%5B1%5D=1&field_news_source_type_tid%5B2%5D=2&field_news_source_type_tid%5B3%5D=3&field_news_bias_nid_1%5B1%5D=1&field_news_bias_nid_1%5B2%5D=2&field_news_bias_nid_1%5B3%5D=3&title=&customFilter=1&page={}"
html_parser = "html5lib"
csv_header = [
    "source",
    "label",
    "agree",
    "disagree",
    "publisher",
    "site",
]
dump_path = "media-bias.csv"
encoding = "utf-8"
skip_blocked_sites = True

verbose = True  # make it True to see debugging messages
level = logging.DEBUG if verbose else logging.INFO
logging.root.handlers.clear()
logging.basicConfig(
    format="%(levelname)s - %(name)s - %(asctime)s - %(message)s",
    level=level
)


async def get_soup(session, url):
    abs_url = urlparse.urljoin(url_tpl, url)
    text = await (await session.get(abs_url)).text()
#     resp.raise_for_status()
    soup = bs4.BeautifulSoup(text, html_parser)
    return soup


async def get_publisher_url(session, src_url, source_name):
#     import code; code.interact(local={**globals(), **locals()})
    logging.debug("Getting publisher's URL for %r.", source_name)
    soup = await get_soup(session, src_url)
    div = soup.find("div", class_="dynamic-grid")
    if not div:
        return None
    
    url = div.find("a").get("href").strip()
    parsed = urlparse.urlparse(url)
    if not parsed.netloc:
        return None
    
    return url, parsed.netloc


async def save_pages(bias_writer, csvfile):
    async with aiohttp.ClientSession() as session:
        page = 28  # custom page if you want
        while True:
            logging.info("Crawling page %d...", page)
            url = url_tpl.format(page)
            soup = await get_soup(session, url)
            
            pub_coros = []
            extras = []
            table = soup.find("table")
            if not table or "no record" in table.find("tbody").find("tr").text.lower():
                logging.info("Reached empty table -> end of results/pages.")
                break
                
            for row in table.find("tbody").find_all("tr"):
                src_a = row.find("td", class_="source-title").find("a")
                src_url = src_a.get("href")
                source_name = src_a.text
                label_alt = row.find("td", class_="views-field-field-bias-image").find("img").get("alt")
                label = label_alt.split(":")[-1].strip()
                feedback = row.find("td", class_="community-feedback")
                agree = int(feedback.find("span", class_="agree").text)
                disagree = int(feedback.find("span", class_="disagree").text)
                
                extras.append([source_name, label, agree, disagree])
#                 import code; code.interact(local={**globals(), **locals()})
                pub_coros.append(get_publisher_url(session, src_url, source_name))
            
            publisher_details_list = await asyncio.gather(*pub_coros)
            for idx, publisher_details in enumerate(publisher_details_list):
                if not publisher_details:
                    if skip_blocked_sites:
                        continue
                    else:
                        publisher_details = ("", "")

#                 print(source_name, label, f"{agree}/{disagree}")
                bias_writer.writerow(extras[idx] + list(publisher_details))

            page += 1
            csvfile.flush()


async def main():
    with open(dump_path, "w", newline="", encoding=encoding) as csvfile:
        bias_writer = csv.writer(csvfile)
        bias_writer.writerow(csv_header)
        await save_pages(bias_writer, csvfile)
        
        
await main()

INFO - root - 2019-11-24 20:49:11,101 - Crawling page 28...
DEBUG - root - 2019-11-24 20:49:11,529 - Getting publisher's URL for 'Yahoo! The 360'.
DEBUG - root - 2019-11-24 20:49:11,530 - Getting publisher's URL for 'Yes! Magazine'.
INFO - root - 2019-11-24 20:49:12,481 - Crawling page 29...
INFO - root - 2019-11-24 20:49:12,639 - Reached empty table -> end of results/pages.


Some publishers are blocked (no websites offered by AllSides), therefore fewer results in the CSV file.

Now let's find a good way of associating a side with a website in case multiple candidates are available.

In [ ]:
side_dict = {}

with open(dump_path, newline="") as stream:
    reader = csv.reader(stream)
    print(next(reader))
    
    for row in reader:
        side_dict.setdefault(row[5], []).append((row[0], row[1], row[2]))

for site, sides in side_dict.items():
    if len(sides) > 1:
        print(site, sides)